# IIC3724-1: Reconocimiento de Patrones
# Tarea 2: Reconocimiento facial
## Agustín Urrutia
Fecha de entrega: 15 de mayo

Mucho del código usado fue copiado o inspirado en el código de actividades y material subido por el profesor.

El script se divide en: 
* Setup, que instala e importa las dependencias y archivos que serán usados. Debe correrse en su totalidad.
* Desarrollo de la tarea, que incluye la extracción de características, explicaciones de por qué se tomaron algunas decisiones y para cada caso se muestran los resultados obtenidos. Es de esta parte que se obtienen los datos guardados en [npy_tarea2_patrones](https://www.dropbox.com/s/k3jot30r61d02ai/npy_tarea2_patrones.zip?dl=0)
* Resultados, solo se importan los datos ya extraidos y filtrados. Para correrlo basta con correr Setup.

### Restricciónes y requisitos

* Incluir al menos tres soluciones distintas y reportar los *accuracies* de cada una de ellas. Señalar mejor método adjuntando diagrama de bloques.
* Guardar características en un archivo para no tener que extraerlas desde cero en cada ejecución.
* Las características extraídas deberán ser seleccionadas y transformadas de tal forma de obtener como máximo un total de no más de 120 características.

Para el desarrollo de la tarea se elegió usar 3 métodos distintos. El primero es la solución más simple usando los extractores de características usados en las actividades manteniendo los parámetros igual a como se usaron allí y se eligieron los atributos que sumasen menos de 120 sin aplicar ningún filtro adicional. Para la segunda estrategia, nuevamente se opta por no usar filtros adicionales, pero a diferencia de lo anterior, se estuvo "jugando" con los parámetros de los extactores de características para obtener algo que resultase con un buen *accuracy*. Finalmente, en la última estrategia, "jugó" nuevamente con los parámetros, pero adicionalmente se usó un método que filtrase características para mejorar los resultados y mantener así también la restricción de no usar más de 120 atributos al clasificar.

## Setup

### Instalación e importación de librerias

In [1]:
!git clone https://github.com/domingomery/balu3
!pip install ./balu3

Cloning into 'balu3'...
remote: Enumerating objects: 261, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 261 (delta 152), reused 177 (delta 74), pack-reused 0
Receiving objects: 100% (261/261), 68.46 KiB | 1.96 MiB/s, done.
Resolving deltas: 100% (152/152), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./balu3
  Preparing metadata (setup.py) ... done
  Created wheel for balu3: filename=balu3-1.0-py3-none-any.whl size=43704 sha256=e1b4ce0a4e571cdac00e91eea58d26034eb02e54980086dc3e73d0058e6dab5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-vdc_m5kj/wheels/9e/f6/a9/3d39da83a3504331129d706f5b04ceb3dbf0ec0f313cdb8fcf
Successfully built balu3


In [2]:
# Librerias numéricas y gráficas
import numpy as np
import matplotlib.pyplot as plt

# Barra de progreso
from   tqdm.auto       import tqdm

# Balu3
from   balu3.cl.basics import SplitTrainTest 
from   balu3.cl.basics import ClassifierKNN  
from   balu3.fx.chr    import lbp, haralick, gabor, hog
from   balu3.io.misc   import imageload
from   balu3.fs.sel    import jfisher,sfs,exsearch, clean
from   balu3.ft.trans  import pca
from   balu3.ft.norm   import minmax

### Cargar imágenes

In [3]:
# Descarga de todas las imágenes 
!wget https://www.dropbox.com/s/3slh34iulgc8nir/FWM.zip
!unzip FWM.zip
print("Done")

--2023-05-16 12:37:15--  https://www.dropbox.com/s/3slh34iulgc8nir/FWM.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/3slh34iulgc8nir/FWM.zip [following]
--2023-05-16 12:37:15--  https://www.dropbox.com/s/raw/3slh34iulgc8nir/FWM.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc49f671213abd96899d6ff68a2b.dl.dropboxusercontent.com/cd/0/inline/B8LGFvJZ7r7hDTBICPCiP4EcUqLqfEKHBSjuyok0Ee28ywjVQtzjgEvbYeNjYPukNpzmULoLlU4m4wp1zGfGB7-n3HqTIj0CrXd9S402Hk25QkG6LKluKthgd74t67yjQI9s3bwBjNn2lOHjkOrBCAwn_IOX0uXTcpI3JiBjvHrF9g/file# [following]
--2023-05-16 12:37:15--  https://uc49f671213abd96899d6ff68a2b.dl.dropboxusercontent.com/cd/0/inline/B8LGFvJZ7r7hDTBICPCiP4EcUqLqfEKHBSjuyok0Ee28ywjVQtzjgEvbYeNjYPukNpzmULoLlU4m4wp1zGfGB7-n3HqTIj

### Importar features 

In [4]:
!wget https://www.dropbox.com/s/k3jot30r61d02ai/npy_tarea2_patrones.zip
!unzip npy_tarea2_patrones.zip

--2023-05-16 12:37:17--  https://www.dropbox.com/s/k3jot30r61d02ai/npy_tarea2_patrones.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/k3jot30r61d02ai/npy_tarea2_patrones.zip [following]
--2023-05-16 12:37:17--  https://www.dropbox.com/s/raw/k3jot30r61d02ai/npy_tarea2_patrones.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccbcea3d1bbd3bd5d2357114479.dl.dropboxusercontent.com/cd/0/inline/B8KnMPDrfi33hybkEwU2WeIQpzHvYP9VfoWuvQBPAYAC2Hn_fZWmJt4Yp5dlu94sOrAbqqq6y-ZXtGeSobsTGmiBLSYDMlQamOEvRqgoklvO0UWaDIECliOQ4mQ21xMSt-5krL9kRi81tJRUvartYidZx7R2Z7tuw0e5qs9xvNDnqA/file# [following]
--2023-05-16 12:37:17--  https://uccbcea3d1bbd3bd5d2357114479.dl.dropboxusercontent.com/cd/0/inline/B8KnMPDrfi33hybkEwU2WeIQpzHvYP9VfoWuvQBPAYAC2H

## Desarrollo de la tarea

### Recordatorio personal para los métodos de extracción de características

Recordatorio de características de intensidad:
<!-- ---------------- -->
* LBP: Invariante a iluminación, no a escala ni rotación (a menos de modificar). 

  `f  = def lbp(img,hdiv=a, vdiv=b, mapping='nri_uniform')`

  Se divide la imagen `img` en `a x b` particiones y en cada una de ellas se extrae el descriptor LBP (vector de 59 elementos). En la actividad en hecha, `f` era un vector de `59*a*b` elementos con `a = b = 1`, es decir sólo una partición por imagen.

<!-- ---------------- -->
* Haralick: Mide distribución de pares de pixeles. Matriz de co-ocurrencia.
  
  `f = def haralick(img,distance=d,)`

  se extrae el vector Haralick para una distancia de `d` pixeles. En la actividad trabajamos con `f` un vector de 24 elementos con `d = 3`.

<!-- ---------------- -->
* Gabor: Se tienen los kernels de Gabor $G_{i,j}$ (matrices). Se hace la convolución del kernel con la imagen para cada uno de los 64 kernels, luego se suman los valores de cada convolución, creando un vector de 64 características
  
  `f = gabor(img,rotations=r, dilations=s)`

  se extrae el vector Gabor para `r` rotaciones y `s` escalas. En la actividad trabajamos con `f` un vector de `r*s+3` elementos con `r = s = 8`.

<!-- ---------------- -->
* HoG: Se calculan los gradientes en diferentes direcciónes 


### Primera estrategia: *naive approach*

Viendo el tamaño de los clasificadores, utilizando los parámetros que se utilizaron en clases, veo qué combinación de ellos me da un clasificador con menos de 120 características y que de un buen resultado.

In [5]:
# Clases y sus tamaños
K = 100
N = 5
IMG_SIZE = 128

### Parámetros de las funciones de extracción de características ###
  # LBP
a = 1
b = 1
  # Haralick
har_d = 3
  # Gabor
gab_r = 8
gab_d = 8
  # HoG
hog_img_division = 1
hog_img_size = IMG_SIZE // hog_img_division
hog_orientation = 9

# Tamaño de las matrices de características
Mlbp = 59 * a * b
Mhar = 24
Mgab = gab_r * gab_d + 3
Mhog = hog_orientation * hog_img_division ** 2

print(f""" Tamaños
Mlbp: {Mlbp}
Mhar: {Mhar}
Mgab: {Mgab}
Mhog: {Mhog}""")


 Tamaños
Mlbp: 59
Mhar: 24
Mgab: 67
Mhog: 9


A priori, podemos ver que no podremos crear un clasificador que incluya todas las características. Pero hay otras distintas combinaciones que sí serán posibles:
* Mlbp + Mhar + Mhog
* Mhar + Mgab + Mhog
* o cualquier subconjunto de ellos

In [6]:
# Construcción de vectores y matrices
Xlbp = np.zeros(( K * N, Mlbp))
Xhar = np.zeros(( K * N, Mhar))
Xgab = np.zeros(( K * N, Mgab))
Xhog = np.zeros(( K * N, Mhog))

y = np.zeros((K * N), 'int')

t = 0

print('Cargando imagenes y extrayendo características...')
for j in tqdm(range(K)):            # para cada clase
  for i in range(N):
    # Lectura de la imagen
    clase     = j+1
    num_img   = i+1
    img       = 255*imageload('FWM/face_', clase, 3, num_img, 3) 

    # LBP clásico de 1x1 particiones
    Xlbp[t,:] = lbp(img,hdiv=a, vdiv=b, mapping='nri_uniform')

    # Haralick con distancia = har_d
    Xhar[t,:] = haralick(img,distance=har_d)

    # Gabor con gab_r rotaciones y gab_d escalas
    # Dado que finalmente no se usó, se va a dejar comentada la línea de 
    # la extracción de características con Gabor
    # Xgab[t,:] = gabor(img,rotations=gab_r, dilations=gab_d)

    # HoG
    Xhog[t,:] = hog(img, orientations=9, pixels_per_cell=(hog_img_size,hog_img_size),cells_per_block=(1,1),norm=False)

    # la muestra t pertenece a la clase j
    y[t] = j 
    
    t = t+1

Cargando imagenes y extrayendo características...


  0%|          | 0/100 [00:00<?, ?it/s]

In [7]:
# Matriz de características
X = np.concatenate(( Xlbp, Xhog, Xhar ), axis=1)
print(X.shape)

(500, 92)


**En la siguiente línea se puede encontrar la matriz de características para ser importada y usada directamente sin tener que correr lo anterior**. Se debe descomentar.

In [8]:
np.save('feature_matrix_1.npy', X)
np.save('y_vector_1.npy', y)
# X = np.load('npy_tarea2_patrones/feature_matrix_1.npy')
# y = np.load('npy_tarea2_patrones/y_vector_1.npy')

In [9]:
# Matrices y vectores de entrenamiento y testeo
ntrain = 4
Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,ntrain)

In [10]:
acc1, C1 = ClassifierKNN(Xtrain,ytrain,Xtest,ytest)

Entrenando con 400 muestras y probando con 100 muestras (de 92 características)...
Testing Accuracy = 92.0%
Matriz de Confusión:
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


Utilizando LBP con a=b=1; HoG con 9 orientaciones y sin dividir la imagen; y Haralick con distancia d=3, se obtuvo un accuracy del 92%. Esta es la respuesta más básica sin hacer ninguna selección inteligente de las características (más allá de la restricción de <=120), pero que funciona bastante bien.




### Manipulación de los parámetros

Se alcanzó una accuracy del 92% sin aplicar ninguna estrategia más allá de cumplir las restricciones de la cantidad de características y usando los parámetros de las actividades. Para el siguiente método, se jugó con los parámetros de las funciones de extracción de características. De este modo, se eligió el método que continuó cumpliendo con que el total de ellas fuese menor al límite de las 120 características. 

Tras el estudio de los diferentes combinaciones hechas, se llegó a que LBP por sí solo, sin combinarlo con otras características, era suficientemente bueno al cambiar sus parámetros `hdiv=1` y `vdiv=2`. Se eliminó la parte del código que finalmente no se usó.

In [11]:
# Clases y sus tamaños
K = 100
N = 5

### Parámetros de las funciones de extracción de características ###
  # LBP
a = 1
b = 2

Mlbp = 59 * a * b

print(f""" Tamaños
Mlbp: {Mlbp}""")

 Tamaños
Mlbp: 118


In [12]:
# Construcción de vectores y matrices
Xlbp = np.zeros(( K * N, Mlbp))

y = np.zeros((K * N), 'int')

t = 0

print('Cargando imagenes y extrayendo características...')
for j in tqdm(range(K)):            # para cada clase
  for i in range(N):
    # Lectura de la imagen
    clase     = j+1
    num_img   = i+1
    img       = 255*imageload('FWM/face_', clase, 3, num_img, 3) 

    # LBP clásico de 1x1 particiones
    Xlbp[t,:] = lbp(img,hdiv=a, vdiv=b, mapping='nri_uniform')

    # la muestra t pertenece a la clase j
    y[t] = j 
    
    t = t+1

Cargando imagenes y extrayendo características...


  0%|          | 0/100 [00:00<?, ?it/s]

**En la siguiente línea se puede encontrar la matriz de características para ser importada y usada directamente sin tener que correr lo anterior**. Se debe descomentar.

In [13]:
np.save('feature_matrix_2.npy', Xlbp)
np.save('y_vector_2.npy', y)
# Xlbp = np.load('npy_tarea2_patrones/feature_matrix_2.npy')
# ylbp = np.load('npy_tarea2_patrones/y_vector_2.npy')

In [14]:
# Matrices y vectores de entrenamiento y testeo
ntrain = 4
Xtrain, ytrain, Xtest, ytest = SplitTrainTest(Xlbp, y, ntrain)

acc2, C2 = ClassifierKNN(Xtrain, ytrain, Xtest, ytest)

Entrenando con 400 muestras y probando con 100 muestras (de 118 características)...
Testing Accuracy = 97.0%
Matriz de Confusión:
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


Usando solo LBP con `hdiv=1` y `vdiv=2` se obtuvo resultados mucho mejores que en el apratado anterior. En primer lugar, el tiempo para la extracción de características fue mucho menor. Pero, lo más importante, fue que la accuracy mejoró de un 92% a un 97%. Cabe mencionar que al cambiar el tamaño de `ntrain` los resultados siguen siendo bastante buenos (`ntrain=3` tiene un 91% de accuracy). 

### Selección de características

Ya alcanzamos una acurracy del 97% sin elección inteligente de las características. Ahora trataremos combinaremos algunos métodos, utilizando algunos parámetros distintos, luego limpiaremos las características (método clean) y finalmente seleccionaremos mediante SFS un set más pequeño de ellas para el entrenamiento y testeo. Veremos si es posible mantener el accuracy anterior pero bajando la dimensionalidad.

In [15]:
# Clases y sus tamaños
K = 100
N = 5
IMG_SIZE = 128

### Parámetros de las funciones de extracción de características ###
  # LBP
a = 2
b = 2
  # Haralick
har_d = 3
  # Gabor
gab_r = 8
gab_d = 8
  # HoG
hog_img_division = 2
hog_img_size = IMG_SIZE // hog_img_division
hog_orientation = 36

Mlbp = 59 * a * b
Mhar = 24
Mgab = gab_r * gab_d + 3
Mhog = hog_orientation * hog_img_division ** 2

print(f""" Tamaños
Mlbp: {Mlbp}
Mhar: {Mhar}
Mgab: {Mgab}
Mhog: {Mhog}""")

 Tamaños
Mlbp: 236
Mhar: 24
Mgab: 67
Mhog: 144


In [16]:
# Construcción de vectores y matrices
Xlbp = np.zeros(( K * N, Mlbp))
Xhar = np.zeros(( K * N, Mhar))
Xgab = np.zeros(( K * N, Mgab))
Xhog = np.zeros(( K * N, Mhog))

y = np.zeros((K * N), 'int')

t = 0

print('Cargando imagenes y extrayendo características...')
for j in tqdm(range(K)):            # para cada clase
  for i in range(N):
    # Lectura de la imagen
    clase     = j+1
    num_img   = i+1
    img       = 255*imageload('FWM/face_', clase, 3, num_img, 3) 

    # LBP clásico de 1x1 particiones
    Xlbp[t,:] = lbp(img,hdiv=a, vdiv=b, mapping='nri_uniform')

    # # Haralick con distancia = har_d
    Xhar[t,:] = haralick(img,distance=har_d)

    # # Gabor con gab_r rotaciones y gab_d escalas
    Xgab[t,:] = gabor(img,rotations=gab_r, dilations=gab_d)

    # # HoG
    Xhog[t,:] = hog(img, orientations=hog_orientation, pixels_per_cell=(hog_img_size,hog_img_size),cells_per_block=(2,2),norm=False)

    # la muestra t pertenece a la clase j
    y[t] = j 
    
    t = t+1

Cargando imagenes y extrayendo características...


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
X = np.concatenate((Xlbp, Xhog, Xhar, Xgab), axis=1)

**En la siguiente línea se puede encontrar la matriz de características para ser importada y usada directamente sin tener que correr lo anterior**. Se debe descomentar.

In [ ]:
np.save('feature_matrix_3.npy', X)
np.save('y_vector_3.npy', y)
# X = np.load('npy_tarea2_patrones/feature_matrix_3.npy')
# y = np.load('npy_tarea2_patrones/y_vector_3.npy')

Solo para probar (no cumple con los requisitos de la tarea), en el siguiente bloque se verá qué tan bien clasifican todos los atributos extraidos juntos.

In [ ]:
Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X, y, ntrain)
acc_ex, C_ex = ClassifierKNN(Xtrain, ytrain, Xtest, ytest)

In [ ]:
features_cleaned = clean(X, show=True)

In [ ]:
Xtrain_cleaned = Xtrain[:,features_cleaned] 

ss = sfs(Xtrain_cleaned, ytrain, 60, show=True)

**En la siguiente línea se puede encontrar el vector con el índice de las características para ser importada y usada directamente sin tener que correr lo anterior**. Se debe descomentar.

In [ ]:
np.save('feature_selected_3.npy', ss)
# ss = np.load('npy_tarea2_patrones/feature_selected_3.npy')

In [ ]:
Xtrain_ex = Xtrain[:,ss]
Xtest_ex  = Xtest[:,ss] 

acc3, C3 = ClassifierKNN(Xtrain_ex, ytrain, Xtest_ex, ytest)

Habiendo probado con varias combinaciones y quedandonos arbitrariamente con solo 60 características, tras limpiar y filtrar con SFS, se obtuvo un accuracy del 95%. Este resultado es peor y tomó más tiempo que el obtenido anteriormente, pero **talvés** pueda ser mejor en otros contextos ya que nos podría ayudar a evitar la *maldición de la dimensionalidad* (es solo una hipótesis)

## Resultados

In [17]:
ntrain = 4

Método 1

In [44]:
X = np.load('npy_tarea2_patrones/feature_matrix_1.npy')
y = np.load('npy_tarea2_patrones/y_vector_1.npy')

Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,ntrain)

acc1, C1 = ClassifierKNN(Xtrain,ytrain,Xtest,ytest)

Entrenando con 400 muestras y probando con 100 muestras (de 92 características)...
Testing Accuracy = 92.0%
Matriz de Confusión:
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


Método 2

In [38]:
Xlbp = np.load('npy_tarea2_patrones/feature_matrix_2.npy')
ylbp = np.load('npy_tarea2_patrones/y_vector_2.npy')

Xtrain,ytrain,Xtest,ytest = SplitTrainTest(Xlbp,ylbp,ntrain)

acc2, C2 = ClassifierKNN(Xtrain,ytrain,Xtest,ytest)

Entrenando con 400 muestras y probando con 100 muestras (de 118 características)...
Testing Accuracy = 97.0%
Matriz de Confusión:
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


Método 3

In [33]:
X = np.load('npy_tarea2_patrones/feature_matrix_3.npy')
y = np.load('npy_tarea2_patrones/y_vector_3.npy')

Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X, y, ntrain)

ss = np.load('npy_tarea2_patrones/feature_selected_3.npy')

Xtrain_ex = Xtrain[:,ss]
Xtest_ex  = Xtest[:,ss] 

acc3, C3 = ClassifierKNN(Xtrain_ex, ytrain, Xtest_ex, ytest)

Entrenando con 400 muestras y probando con 100 muestras (de 60 características)...
Testing Accuracy = 95.0%
Matriz de Confusión:
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
